[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20XlmRoBertaForTokenClassification.ipynb)

## Import XlmRoBertaForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.3.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import XLM-RoBERTa models trained/fine-tuned for token classification via `XLMRobertaForTokenClassification` or `TFXLMRobertaForTokenClassification`. These models are usually under `Token Classification` category and have `xlm-roberta` in their labels
- Reference: [TFXLMRobertaForTokenClassification](https://huggingface.co/transformers/model_doc/xlmroberta.html#tfxlmrobertafortokenclassification)
- Some [example models](https://huggingface.co/models?filter=xlm-roberta&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.1` version and Transformers on `4.10.0`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- XLMRobertaTokenizer requires the `SentencePiece` library, so we install that as well

In [ ]:
!pip install -q transformers==4.10.0 tensorflow==2.4.1 sentencepiece

     |████████████████████████████████| 394.3 MB 13 kB/s 
     |████████████████████████████████| 1.2 MB 63.9 MB/s 
     |████████████████████████████████| 636 kB 57.4 MB/s 
     |████████████████████████████████| 895 kB 61.0 MB/s 
     |████████████████████████████████| 3.3 MB 48.6 MB/s 
     |████████████████████████████████| 52 kB 1.8 MB/s 
     |████████████████████████████████| 2.9 MB 41.5 MB/s 
     |████████████████████████████████| 462 kB 41.6 MB/s 
     |████████████████████████████████| 3.8 MB 47.9 MB/s 


- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [wpnbos/xlm-roberta-base-conll2002-dutch](https://huggingface.co/wpnbos/xlm-roberta-base-conll2002-dutch) model from HuggingFace as an example
- In addition to `TFXLMRobertaForTokenClassification` we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFXLMRobertaForTokenClassification, XLMRobertaTokenizer 

MODEL_NAME = 'wpnbos/xlm-roberta-base-conll2002-dutch'

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFXLMRobertaForTokenClassification.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFXLMRobertaForTokenClassification.from_pretrained(MODEL_NAME, from_pt=True)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/356 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForTokenClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForTokenClassification for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

INFO:tensorflow:Assets written to: ./wpnbos/xlm-roberta-base-conll2002-dutch/saved_model/1/assets


INFO:tensorflow:Assets written to: ./wpnbos/xlm-roberta-base-conll2002-dutch/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 1084108
-rw-r--r-- 1 root root       1130 Sep 20 12:19 config.json
drwxr-xr-x 3 root root       4096 Sep 20 12:19 saved_model
-rw-r--r-- 1 root root 1110111488 Sep 20 12:19 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 8096
drwxr-xr-x 2 root root    4096 Sep 20 12:19 assets
-rw-r--r-- 1 root root 8278963 Sep 20 12:19 saved_model.pb
drwxr-xr-x 2 root root    4096 Sep 20 12:19 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 4960
-rw-r--r-- 1 root root 5069051 Sep 20 12:17 sentencepiece.bpe.model
-rw-r--r-- 1 root root     239 Sep 20 12:17 special_tokens_map.json
-rw-r--r-- 1 root root     633 Sep 20 12:17 tokenizer_config.json


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `sentencepiece.bpe.model` file from the tokenizer
- all we need is to copy `sentencepiece.bpe.model` file into `saved_model/1/assets` which Spark NLP will look for
- in addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

# let's copy sentencepiece.bpe.model file to saved_model/1/assets
!cp {MODEL_NAME}_tokenizer/sentencepiece.bpe.model {asset_path}

In [ ]:
# get label2id dictionary 
labels = model.config.label2id
# sort the dictionary based on the id
labels = sorted(labels, key=labels.get)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [ ]:
!ls -l {asset_path}

total 4956
-rw-r--r-- 1 root root      71 Sep 20 12:26 labels.txt
-rw-r--r-- 1 root root 5069051 Sep 20 12:26 sentencepiece.bpe.model


## Import and Save XlmRoBertaForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `XlmRoBertaForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlmRoBertaForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` only accepts local paths and not distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. That is why we use `write.save` so we can use `.load()` from any file systems



In [ ]:
from sparknlp.annotator import *

tokenClassifier = XlmRoBertaForTokenClassification\
  .loadSavedModel('{}/saved_model/1'.format(MODEL_NAME), spark)\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your XlmRoBertaForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 1096760
drwxr-xr-x 4 root root       4096 Sep 20 12:31 fields
drwxr-xr-x 2 root root       4096 Sep 20 12:31 metadata
-rw-r--r-- 1 root root 1117998527 Sep 20 12:32 xlm_roberta_classification_tensorflow
-rw-r--r-- 1 root root    5069051 Sep 20 12:32 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlmRoBertaForTokenClassification model 😊 

In [ ]:
tokenClassifier_loaded = XlmRoBertaForTokenClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("ner")

In [ ]:
tokenClassifier_loaded.getCaseSensitive()

True

That's it! You can now go wild and use hundreds of `XlmRoBertaForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀 
